##Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# from sklearn import datasets
from sklearn import preprocessing

from sklearn import metrics
from scipy import stats
import scipy.stats as ss

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor

from itertools import product

##Import dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ST4052/project2/datasets/final_merged_train.csv')
train.head()

,Unnamed: 0,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,...,debit_lag,fraudcount_lag1,fraudcount_lag2,fraudcount_lag3,fraudcount_lag4,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9
0,5597,462,TRANSFER,151994.59,151994.59,0.0,0.00,0.00,1,0,...,1474.600625,16.0,16.0,8.0,6.0,10.0,18.0,10.0,6.0,8.0
1,5598,462,CASH_OUT,151994.59,151994.59,0.0,0.00,151994.59,1,0,...,1474.600625,16.0,16.0,8.0,6.0,10.0,18.0,10.0,6.0,8.0
2,5599,462,TRANSFER,4342582.37,4342582.37,0.0,0.00,0.00,1,0,...,1474.600625,16.0,16.0,8.0,6.0,10.0,18.0,10.0,6.0,8.0
3,5600,462,CASH_OUT,4342582.37,4342582.37,0.0,5398482.58,9741064.96,1,0,...,1474.600625,16.0,16.0,8.0,6.0,10.0,18.0,10.0,6.0,8.0
4,5601,462,TRANSFER,67921.66,67921.66,0.0,0.00,0.00,1,0,...,1474.600625,16.0,16.0,8.0,6.0,10.0,18.0,10.0,6.0,8.0


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/ST4052/project2/datasets/final_merged_test.csv')
test.head()

,Unnamed: 0,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,...,debit_lag,fraudcount_lag1,fraudcount_lag2,fraudcount_lag3,fraudcount_lag4,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9
0,3638,678,TRANSFER,1536733.52,1536733.52,0.0,0.00,0.00,1,0,...,1318.25,8.0,8.0,10.0,14.0,12.0,8.0,18.0,12.0,8.0
1,3639,678,CASH_OUT,1536733.52,1536733.52,0.0,244150.15,1780883.67,1,0,...,1318.25,8.0,8.0,10.0,14.0,12.0,8.0,18.0,12.0,8.0
2,3640,678,TRANSFER,194876.96,194876.96,0.0,0.00,0.00,1,0,...,1318.25,8.0,8.0,10.0,14.0,12.0,8.0,18.0,12.0,8.0
3,3641,678,CASH_OUT,194876.96,194876.96,0.0,148344.11,343221.07,1,0,...,1318.25,8.0,8.0,10.0,14.0,12.0,8.0,18.0,12.0,8.0
4,3642,678,TRANSFER,749316.70,749316.70,0.0,0.00,0.00,1,0,...,1318.25,8.0,8.0,10.0,14.0,12.0,8.0,18.0,12.0,8.0


In [ ]:
x_train = train.drop(['Unnamed: 0', 'isFraud'], axis=1)
y_train = train['isFraud']

x_test = test.drop(['Unnamed: 0', 'isFraud'], axis=1)
y_test = test['isFraud']

###Encoding type parameter

In [ ]:
ohe = OneHotEncoder(sparse=False)

In [ ]:
type_train = ohe.fit_transform(x_train[['type']])
type_test = ohe.fit_transform(x_test[['type']])

In [ ]:
# x_train[x_train['type'] == 'PAYMENT']

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,fraud_count,payment_lag,...,debit_lag,fraudcount_lag1,fraudcount_lag2,fraudcount_lag3,fraudcount_lag4,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9
54,466,PAYMENT,37068.66,242348.00,205279.34,0.0,0.0,0,6,0.0,...,2443.058000,12.0,16.0,10.0,16.0,16.0,16.0,8.0,6.0,10.0
56,466,PAYMENT,781.47,20949.00,20167.53,0.0,0.0,0,6,0.0,...,2443.058000,12.0,16.0,10.0,16.0,16.0,16.0,8.0,6.0,10.0
58,466,PAYMENT,887.10,9982.00,9094.90,0.0,0.0,0,6,0.0,...,2443.058000,12.0,16.0,10.0,16.0,16.0,16.0,8.0,6.0,10.0
62,466,PAYMENT,44190.85,280.00,0.00,0.0,0.0,0,6,0.0,...,2443.058000,12.0,16.0,10.0,16.0,16.0,16.0,8.0,6.0,10.0
63,466,PAYMENT,3119.88,320002.00,316882.12,0.0,0.0,0,6,0.0,...,2443.058000,12.0,16.0,10.0,16.0,16.0,16.0,8.0,6.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272689,667,PAYMENT,7666.00,19854.00,12188.00,0.0,0.0,0,4,0.0,...,1346.408571,6.0,10.0,20.0,10.0,2.0,8.0,12.0,10.0,14.0
272694,667,PAYMENT,2541.41,30722.19,28180.79,0.0,0.0,0,4,0.0,...,1346.408571,6.0,10.0,20.0,10.0,2.0,8.0,12.0,10.0,14.0
272698,667,PAYMENT,13551.24,41657.00,28105.76,0.0,0.0,0,4,0.0,...,1346.408571,6.0,10.0,20.0,10.0,2.0,8.0,12.0,10.0,14.0
272715,667,PAYMENT,6922.19,20757.00,13834.81,0.0,0.0,0,4,0.0,...,1346.408571,6.0,10.0,20.0,10.0,2.0,8.0,12.0,10.0,14.0


In [ ]:
# type_[54]

array([0., 0., 0., 1., 0.])

In [ ]:
x_train[['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']] = type_train
x_test[['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']] = type_test

In [ ]:
x_train

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,fraud_count,payment_lag,...,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,462,TRANSFER,151994.59,151994.59,0.00,0.00,0.00,0,16,10257.436286,...,10.0,18.0,10.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0
1,462,CASH_OUT,151994.59,151994.59,0.00,0.00,151994.59,0,16,10257.436286,...,10.0,18.0,10.0,6.0,8.0,0.0,1.0,0.0,0.0,0.0
2,462,TRANSFER,4342582.37,4342582.37,0.00,0.00,0.00,0,16,10257.436286,...,10.0,18.0,10.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0
3,462,CASH_OUT,4342582.37,4342582.37,0.00,5398482.58,9741064.96,0,16,10257.436286,...,10.0,18.0,10.0,6.0,8.0,0.0,1.0,0.0,0.0,0.0
4,462,TRANSFER,67921.66,67921.66,0.00,0.00,0.00,0,16,10257.436286,...,10.0,18.0,10.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272719,667,CASH_OUT,120935.88,25718.40,0.00,143951.42,264887.30,0,4,0.000000,...,2.0,8.0,12.0,10.0,14.0,0.0,1.0,0.0,0.0,0.0
272720,667,CASH_OUT,159993.98,302141.00,142147.02,946781.49,1106775.47,0,4,0.000000,...,2.0,8.0,12.0,10.0,14.0,0.0,1.0,0.0,0.0,0.0
272721,667,CASH_OUT,184270.30,10500.00,0.00,864834.69,1049104.99,0,4,0.000000,...,2.0,8.0,12.0,10.0,14.0,0.0,1.0,0.0,0.0,0.0
272722,667,PAYMENT,13798.36,0.00,0.00,0.00,0.00,0,4,0.000000,...,2.0,8.0,12.0,10.0,14.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
x_test

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,fraud_count,payment_lag,...,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,678,TRANSFER,1536733.52,1536733.52,0.0,0.00,0.00,0,6,0.0,...,12.0,8.0,18.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0
1,678,CASH_OUT,1536733.52,1536733.52,0.0,244150.15,1780883.67,0,6,0.0,...,12.0,8.0,18.0,12.0,8.0,0.0,1.0,0.0,0.0,0.0
2,678,TRANSFER,194876.96,194876.96,0.0,0.00,0.00,0,6,0.0,...,12.0,8.0,18.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0
3,678,CASH_OUT,194876.96,194876.96,0.0,148344.11,343221.07,0,6,0.0,...,12.0,8.0,18.0,12.0,8.0,0.0,1.0,0.0,0.0,0.0
4,678,TRANSFER,749316.70,749316.70,0.0,0.00,0.00,0,6,0.0,...,12.0,8.0,18.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65939,743,CASH_OUT,339682.13,339682.13,0.0,0.00,339682.13,0,8,0.0,...,10.0,10.0,14.0,12.0,8.0,0.0,1.0,0.0,0.0,0.0
65940,743,TRANSFER,6311409.28,6311409.28,0.0,0.00,0.00,0,8,0.0,...,10.0,10.0,14.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0
65941,743,CASH_OUT,6311409.28,6311409.28,0.0,68488.84,6379898.11,0,8,0.0,...,10.0,10.0,14.0,12.0,8.0,0.0,1.0,0.0,0.0,0.0
65942,743,TRANSFER,850002.52,850002.52,0.0,0.00,0.00,0,8,0.0,...,10.0,10.0,14.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
x_train.drop('type', axis=1, inplace=True)
x_test.drop('type', axis=1, inplace=True)

In [ ]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(272724, 27)
(272724,)
(65944, 27)
(65944,)


In [ ]:
x_train.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,fraud_count,payment_lag,cashin_lag,...,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,462,151994.59,151994.59,0.0,0.00,0.00,0,16,10257.436286,115640.904,...,10.0,18.0,10.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0
1,462,151994.59,151994.59,0.0,0.00,151994.59,0,16,10257.436286,115640.904,...,10.0,18.0,10.0,6.0,8.0,0.0,1.0,0.0,0.0,0.0
2,462,4342582.37,4342582.37,0.0,0.00,0.00,0,16,10257.436286,115640.904,...,10.0,18.0,10.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0
3,462,4342582.37,4342582.37,0.0,5398482.58,9741064.96,0,16,10257.436286,115640.904,...,10.0,18.0,10.0,6.0,8.0,0.0,1.0,0.0,0.0,0.0
4,462,67921.66,67921.66,0.0,0.00,0.00,0,16,10257.436286,115640.904,...,10.0,18.0,10.0,6.0,8.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
x_test.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,fraud_count,payment_lag,cashin_lag,...,fraudcount_lag5,fraudcount_lag6,fraudcount_lag7,fraudcount_lag8,fraudcount_lag9,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,678,1536733.52,1536733.52,0.0,0.00,0.00,0,6,0.0,164712.668158,...,12.0,8.0,18.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0
1,678,1536733.52,1536733.52,0.0,244150.15,1780883.67,0,6,0.0,164712.668158,...,12.0,8.0,18.0,12.0,8.0,0.0,1.0,0.0,0.0,0.0
2,678,194876.96,194876.96,0.0,0.00,0.00,0,6,0.0,164712.668158,...,12.0,8.0,18.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0
3,678,194876.96,194876.96,0.0,148344.11,343221.07,0,6,0.0,164712.668158,...,12.0,8.0,18.0,12.0,8.0,0.0,1.0,0.0,0.0,0.0
4,678,749316.70,749316.70,0.0,0.00,0.00,0,6,0.0,164712.668158,...,12.0,8.0,18.0,12.0,8.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
frame_train = [x_train,y_train]
train_df = pd.concat(frame_train,axis = 1)

train_df.to_csv('final_merged_encoded_train.csv')


In [ ]:
frame_test = [x_test,y_test]
test_df = pd.concat(frame_test,axis = 1)

test_df.to_csv('final_merged_encoded_test.csv')

##Picked variables

In [ ]:
# amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest, fraud_count, cashin_lag, cashout_lag, transfer_lag	, debit_lag, fraudcount_lag3, fraudcount_lag5, 
# fraudcount_lag7

In [ ]:
## 23 variables
frame_train_feature_fraudcount_lag3 = [x_train_feature_fraudcount_lag3, y_train]
train_feature_fraudcount_lag3 = pd.concat(frame_train_feature_fraudcount_lag3, axis=1)
train_feature_fraudcount_lag3.to_csv('train_feature_fraudcount_lag3.csv')

## 10 variables
frame_train_feature_payment_lag = [x_train_feature_payment_lag, y_train]
train_feature_payment_lag = pd.concat(frame_train_feature_payment_lag, axis=1)
train_feature_payment_lag.to_csv('train_feature_payment_lag.csv')

In [ ]:
x_test_feature_fraudcount_lag3 = x_test[important_fraudcount_lag3]      # selecting only 23 variables from x_test
x_test_feature_payment_lag = x_test[important_payment_lag]              # selecting only 10 variables from x_test

## 23 variables 
frame_test_feature_fraudcount_lag3 = [x_test_feature_fraudcount_lag3, y_test]
test_feature_fraudcount_lag3 = pd.concat(frame_test_feature_fraudcount_lag3, axis=1)
test_feature_fraudcount_lag3.to_csv('test_feature_fraudcount_lag3.csv')

## 10 variables
frame_test_feature_payment_lag = [x_test_feature_payment_lag, y_test]
test_feature_payment_lag = pd.concat(frame_test_feature_payment_lag, axis=1)
test_feature_payment_lag.to_csv('test_feature_payment_lag.csv')

x_train_feature_fraudcount_lag3 (23 variables) and x_train_feature_payment_lag (10 variables) , 2 dataframes are used to fit the model. 